# Programming Assignment

### 1. Libraries

In [28]:
import re
import networkx as nx


### 2. Read the input and check for a valid regex

In [29]:
# Read the input from the user 
input_regex = r"a?(a|b)*?b"

# Check on the input is a valid regex

re.compile(input_regex)


re.compile(r'a?(a|b)*?b', re.UNICODE)

### 3. Preprocessing 
<P>Remove square brackets and replace it with <b>OR</b>ing</P>
<P>Add concatenation character</P>


In [30]:
def find_square_bracket_ranges(regex):
    ranges = []
    start = -1
    for i, char in enumerate(regex):
        if char == "[":
            if start == -1:
                start = i + 1
        elif char == "]":
            if start != -1:
                ranges.append((start, i))
                start = -1
                
    return ranges


def split_ranges(s: str):
    expanded_string = ""
    it = iter(enumerate(s))
    for i, char in it:
        if i + 1 != len(s) and s[i + 1] == "-":
            temp_str = ""
            start_symbol = char
            end_symbol = s[i + 2]
            ascii_range = ord(end_symbol) - ord(start_symbol) + 1
            for i in range(ascii_range):
                temp_str += chr(ord(start_symbol) + i) + "|"
            expanded_string += temp_str
            [next(it) for _ in range(2)]
        elif char != "-":
            expanded_string += char + "|"

    return expanded_string[:-1]


CONCAT_SYMBOL = "#"


def insert_concat_symbol_2(regex):
    result = ""
    inside_brackets = False
    for i in range(len(regex)):
        if regex[i] == "[":
            inside_brackets = True
        elif regex[i] == "]":
            inside_brackets = False

        if (
            not inside_brackets
            and i < len(regex) - 1
            and regex[i] != "("
            # and regex[i + 1] != "("
            # and regex[i] != ")"
            and regex[i + 1] != ")"
            and regex[i + 1] not in ["?", ".", "*", "|", "+"]
            and regex[i] not in ["?", ".", "*", "|", "+"]
        ):
            result += regex[i] + CONCAT_SYMBOL
        else:
            result += regex[i]
    return result

def preprocess_regex_2(regex):
    # Preprocess ranges
    ranges = find_square_bracket_ranges(regex)
    expanded_ranges = []
    for start, end in ranges:
        expanded_ranges.append(split_ranges(regex[start:end]))
    # Replace ranges with expanded ranges
    inside_square_brackets = False
    inserted_expanded_range = False
    bracket_count = 0
    new_regex = ""

    for char in regex:
        if char == "[":
            bracket_count += 1
            inside_square_brackets = True
            new_regex += "("
        elif char == "]":
            inside_square_brackets = False
            inserted_expanded_range = False
            new_regex += ")"
        elif not inside_square_brackets:
            new_regex += char
        elif inside_square_brackets and not inserted_expanded_range:
            new_regex += expanded_ranges[bracket_count - 1]
            inserted_expanded_range = True
    new_regex = insert_concat_symbol_2(new_regex)
    print(new_regex)
    return new_regex


In [34]:

CONCAT_SYMBOL = "#"


def insert_concat_symbol(regex):
    result = ""
    operators = ["?", ".", "*", "|", "+"]
    for i in range(len(regex)):
        if (
            i < len(regex) - 1
            and regex[i + 1] not in operators
            and regex[i] not in operators
            and regex[i] != "("
            and regex[i + 1] != ")"
        ):
            result += regex[i] + CONCAT_SYMBOL
        else:
            result += regex[i]      
    return result


def preprocess_regex(regex):
    new_regex = ""
    skip = -1
    for i, char in enumerate(regex):
        if i <= skip:
            continue
        if char == "[":
            end = regex.find(']', i)  # Find the end of the bracketed expression
            if end == -1:
                raise ValueError('Mismatched brackets')  # Raise an error if there is no matching close bracket
            # Handle ranges within the bracketed expression
            bracketed = regex[i+1:end]
            if '-' in bracketed:
                new_bracketed = ""
                splited_bracketed = bracketed.split('-')
                for j, _ in enumerate(splited_bracketed):
                    if j + 1 != len(splited_bracketed):
                        range_start = splited_bracketed[j]
                        range_end = splited_bracketed[j + 1]
                        ascii_range = ord(range_end[0]) - ord(range_start[-1])
                        range_symbols = ""
                        for k in range(ascii_range):
                            if k != 0:
                                range_symbols += chr(ord(range_start[-1]) + k) + "|"
                        if j == 0:
                            len_range_start = len(range_start)
                            for k, ch in enumerate(range_start):
                                if k == len_range_start - 1:
                                    new_bracketed += ch
                                    continue
                                new_bracketed += ch + "|"
                        new_bracketed += "|" + range_symbols 
                        for k, ch in enumerate(range_end):
                            if k == 0:
                                new_bracketed += ch
                                continue
                            new_bracketed += "|" + ch
            new_regex += f"({new_bracketed})"  # Treat the entire bracketed expression as a single character
            skip = end      # Skip to the end of the bracketed expression
        else:
            new_regex += char
    new_regex = insert_concat_symbol(new_regex)        
    print(new_regex)
    return new_regex            


### 4. Shunting yard algorithm 
<P>Convert infix regex to postfix regex</P>

In [32]:
def shunting_yard(input_regex):
    # Define precedence
    """
        * : 0 or more repetition
        + : 1 or more repetition
        # : concatenation
        . : Any single character
        | : OR
    """
    precedence = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
    
    precedence = {
    '*': 6,  # 0 or more repetition
    '+': 5,  # 1 or more repetition
    '?': 4,  # optional (exists or not)
    '#': 3,  # concatenation operator
    '.': 2,  # any single character
    '|': 1   # OR operator
    }
    
    # postfix_output queue and operator operator_stack
    postfix_output = []
    operator_stack = []

    # Process each character in the input
    i = 0
    while i < len(input_regex):
        char = input_regex[i]
        if char == '(':
            operator_stack.append(char)
        elif char == ')':
            while operator_stack and operator_stack[-1] != '(':
                postfix_output.append(operator_stack.pop())
            # check operator_stack error if ')' is not have openning '('
            if not operator_stack:
                return None
            operator_stack.pop()  # Remove the '('
        elif char in precedence:
            while operator_stack and operator_stack[-1] in precedence and precedence[char] <= precedence[operator_stack[-1]]:
                postfix_output.append(operator_stack.pop())
            operator_stack.append(char)
        else:
            postfix_output.append(char)
        i += 1

    # Pop remaining operators from the operator_stack to the postfix_output
    while operator_stack:
        # check operator_stack error if there is still '(' in the operator_stack so it won't have a close ')'
        if operator_stack[-1] == '(':
            return None
        postfix_output.append(operator_stack.pop())

    # Return the postfix_output as a string
    return ''.join(postfix_output)


### 5. Convert postfix regex to NFA using thomson's rule

### 6. Convert NFA to DFA using subset construction 


### 7. Convert to minimized DFA

### 8. Visualizations Graphiz

### Testing

In [37]:


ff = "(asjnd[0-9])|(h4d2*|33+as449d|[2-9a-z(az)*])"
ss = "BAM[a-dA-Ds7-9](ABC)[x-z]?ABC(a)[a-b](FA)(BC)?"

# input_rege = preprocess_regex_2(ss)
input_regex = preprocess_regex(ss)
# print(input_regex == input_rege)


print(shunting_yard(input_regex))



B#A#M#(a|b|c|d|A|B|C|D|s|7|8|9)#(A#B#C)#(x|y|z)?A#B#C#(a)#(a|b)#(F#A)#(B#C)?
B#A#M#(a|b|c|d|A|B|C|D|s|7|8|9)#(A#B#C)#(x|y|z)?A#B#C#(a)#(a|b)#(F#A)#(B#C)?
True
BA#M#ab|c|d|A|B|C|D|s|7|8|9|#AB#C##xy|z|A?#B#C#a#ab|#FA##BC#?#
